In [2]:
import requests
import pandas as pd
import numpy as np
import datetime as dt
import json
import pytz

from pandas.io.json import json_normalize
from IPython.display import display
from datetime import datetime
from pandas.io.json import json_normalize
from datetime import date, timedelta
pd.options.mode.chained_assignment = None  #ignores the warning message and code runs faster # default='warn'

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 2000)
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
table = pd.read_csv('gs://aes-datahub-0001-raw/Weather/Dark_Sky/USA/Indianapolis/Historical data/Storm data 2007-2019/location_specific_data/All_Markers_Combined_Storm.csv')

In [ ]:
date_list = list(table.Date.unique())

In [ ]:
def ws_historical_data(dates, lat, long, period='day', fields='all'):
    '''
    If duration is more than 1 year separate calls should be used
    Timestamp should be converted to ISO 8601 format
    Docstring with examples and function return values:
    
    Input :
    start - (%Y-%m-%d) format
    end - (%Y-%m-%d) format
    lat - latitude 
    long - longitude
    period - hour, day (default=day)
    
    Output : return a callection of weather historical data for a latitude/longitude point
    
    '''
    key = 'e721181f854ac2268ee8'
    
    
    master_weather_ = pd.DataFrame()
    
    for date in dates:
        date = pd.to_datetime(date,format='%Y-%m-%d').strftime('%Y-%m-%dT%H:%M:%S')
        weather_ = pd.DataFrame()
        link = 'https://api.weathersource.com/v1/'+key+'/points/'+lat+','+long+'/history.json?period='+period+'&timestamp_between='+date+','+date+'&fields='+fields
        print(link)
        response = requests.get(link)
        json_obj = json.loads(response.content.decode('utf-8'))
        weather_ = json_normalize(json_obj)
        master_weather_ = master_weather_.append(weather_, sort = True)

    return master_weather_

In [ ]:
sites_latitude = {
    'Marker1' : '39.9613','Marker2' : '39.8971','Marker3' : '39.9060','Marker4' : '39.9024','Marker5' : '39.8960','Marker6' : '39.8339',
    'Marker7' : '39.8412','Marker8' : '39.8381','Marker9' : '39.8386','Marker10' : '39.7579','Marker11' : '39.7621','Marker12' : '39.7621',
    'Marker13' : '39.7695','Marker14' : '39.6617','Marker15' : '39.6639','Marker16' : '39.6702','Marker17' : '39.6744','Marker18' : '39.5909',
    'Marker19' : '39.5295','Marker20' : '39.5475'
}

# longitude of the location markers
sites_longitude = {
    'Marker1' : '-86.4034','Marker2' : '-86.3045','Marker3' : '-86.2001','Marker4' : '-86.0738','Marker5' : '-85.9783','Marker6' : '-86.3155',
    'Marker7' : '-86.2056','Marker8' : '-86.0985','Marker9' : '-85.9811','Marker10' : '-86.3155','Marker11' : '-86.2042','Marker12' : '-86.0923',
    'Marker13' : '-85.9708','Marker14' : '-86.2935','Marker15' : '-86.1823','Marker16' : '-86.0669','Marker17' : '-85.9557','Marker18' : '-86.4212',
    'Marker19' : '-86.5874','Marker20' : '-86.2743'
}

key = list(sites_latitude.keys())

data_file = []

# for i in key:
#     data = ws_historical_data(date_list, lat=sites_latitude.get(i), long=sites_longitude.get(i), period='day')
#     data['Location'] = i
#     data_file.append(data)
    
# weather_source_df = pd.concat(data_file)
# weather_source_df.reset_index(drop=True, inplace=True)

In [ ]:
# weather_source_df.to_csv('Weather_Source_All_Marker.csv')

In [ ]:
key

In [ ]:
def getdarkskydatadaily(start,end,loc_name):
    """
    
    Input - start_date, end_date, API KEY from https://darksky.net/dev/register (now closed APPLE has acqired darksky)
    Output -daily level weather data from start date to end date for specified location
    
    """
    api_key ='03f7d7221972645470c7e8c6f35edd85' #Azure
    headers = {'User-Agent': 'Chrome/78and.0.3865.90'}
    http_proxy  = "http://10.245.5.249:8080"
    https_proxy = "https://10.245.5.249:8080"
    ftp_proxy   = "ftp://10.245.5.249:8080"

 

    proxyDict = { 
                "http"  : http_proxy, 
                "https" : https_proxy, 
                "ftp"   : ftp_proxy
                 }
    
    weather_data_daily = pd.DataFrame()
    date_list = []
    
    # get month,year and day from start date
    start_date = dt.datetime.strptime(start, "%Y-%m-%d")
    start_month = start_date.month
    start_year = start_date.year
    start_day = start_date.day
    
    # get month,year and day from end date
    end_date = dt.datetime.strptime(end, "%Y-%m-%d")
    end_month = end_date.month
    end_year = end_date.year
    end_day = end_date.day
    
    # store all date from start date to end date in date list 
    start_date = date(start_year, start_month, start_day)
    end_date = date(end_year, end_month, end_day)
    delta = timedelta(days=1)
    
    while start_date <= end_date:
        date_list.append(start_date.strftime("%Y-%m-%d"))
        start_date += delta
    
    # latitude of the location markers
    sites_latitude = {
        'Marker1' : '39.9613','Marker2' : '39.8971','Marker3' : '39.9060','Marker4' : '39.9024','Marker5' : '39.8960','Marker6' : '39.8339',
        'Marker7' : '39.8412','Marker8' : '39.8381','Marker9' : '39.8386','Marker10' : '39.7579','Marker11' : '39.7621','Marker12' : '39.7621',
        'Marker13' : '39.7695','Marker14' : '39.6617','Marker15' : '39.6639','Marker16' : '39.6702','Marker17' : '39.6744','Marker18' : '39.5909',
        'Marker19' : '39.5295','Marker20' : '39.5475'
    }
    
    # longitude of the location markers
    sites_longitude = {
        'Marker1' : '-86.4034','Marker2' : '-86.3045','Marker3' : '-86.2001','Marker4' : '-86.0738','Marker5' : '-85.9783','Marker6' : '-86.3155',
        'Marker7' : '-86.2056','Marker8' : '-86.0985','Marker9' : '-85.9811','Marker10' : '-86.3155','Marker11' : '-86.2042','Marker12' : '-86.0923',
        'Marker13' : '-85.9708','Marker14' : '-86.2935','Marker15' : '-86.1823','Marker16' : '-86.0669','Marker17' : '-85.9557','Marker18' : '-86.4212',
        'Marker19' : '-86.5874','Marker20' : '-86.2743'
    }
    
    value1 = sites_latitude.get(loc_name)
    value2 = sites_longitude.get(loc_name)
    if(value1 == None and value2 == None):
        print('Unknown Location Name Used: %s',loc_name)
        print('\n')
        print('Use some other Consumer Location or Generation Location Name.. .. ..')
        print('\n')
    else :
        for i in range(len(date_list)):
            #requesting web page
            date_format = '%Y-%m-%dT%H:%M:%S'
            dates = dt.datetime.strptime(date_list[i], '%Y-%m-%d').strftime(date_format)
            links = 'https://api.darksky.net/forecast/'+api_key+'/'+sites_latitude.get(loc_name)+','+sites_longitude.get(loc_name)+','+str(dates)+'?exclude=currently,hourly,alerts?units=us'
            print(links)
            response = requests.get(links,headers=headers,proxies=proxyDict)

 

            # converting into json
            weather = json.loads(response.content.decode('utf-8'))
            # getting it into data frame
            weather_data = json_normalize(weather['daily']['data'])
            weather_data_daily = weather_data_daily.append(weather_data,sort=True)
    
    weather_data_daily.reset_index(drop=True, inplace=True)
    
    return weather_data_daily

In [3]:
start_date = "2018-03-01"
end_date = "2018-03-31"
key = ['Marker1']

darksky_file = []

for i in key:
    darksky_data = getdarkskydatadaily(start_date,end_date,i)
    darksky_data['Location'] = i
    darksky_data.to_csv('Darksky_weather_'+i+'_.csv')
    darksky_file.append(darksky_data)

NameError: name 'getdarkskydatadaily' is not defined

In [5]:
darksky_files = []
for i in range(20):
    darksky_data = pd.read_csv('Darksky_weather_Marker'+str(i+1)+'_.csv')
    darksky_files.append(darksky_data)

darksy_df = pd.concat(darksky_files)
darksy_df.reset_index(drop=True, inplace=True)

In [6]:
darksy_df.head()

,Unnamed: 0,apparentTemperatureHigh,apparentTemperatureHighTime,apparentTemperatureLow,apparentTemperatureLowTime,apparentTemperatureMax,apparentTemperatureMaxTime,apparentTemperatureMin,apparentTemperatureMinTime,cloudCover,dewPoint,humidity,icon,moonPhase,ozone,precipIntensity,precipIntensityMax,precipIntensityMaxTime,precipProbability,precipType,pressure,summary,sunriseTime,sunsetTime,temperatureHigh,temperatureHighTime,temperatureLow,temperatureLowTime,temperatureMax,temperatureMaxTime,temperatureMin,temperatureMinTime,time,uvIndex,uvIndexTime,visibility,windBearing,windGust,windGustTime,windSpeed,Location
0,0,94.20,1562018940,73.56,1562060280,94.20,1562018940,67.20,1561971900,0.22,69.16,0.78,rain,0.97,301.40,0.00,0.02,1562029440.00,0.33,rain,1015.40,Humid throughout the day.,1561976520,1562030400,87.97,1562018340,71.61,1562060040,87.97,1562018340,65.74,1561971840,1561953600,9,1562002920,10.00,287,17.53,1561993560,6.72,Marker1
1,1,95.30,1562091900,73.20,1562142060,95.30,1562091900,73.56,1562060280,0.19,71.76,0.76,rain,0.01,306.60,0.01,0.02,1562072760.00,0.50,rain,1013.00,Humid throughout the day.,1562062920,1562116740,90.19,1562094900,71.38,1562142180,90.19,1562094900,71.61,1562060040,1562040000,9,1562091000,10.00,243,16.72,1562107920,7.84,Marker1
2,2,91.00,1562183160,68.48,1562233560,91.00,1562183160,73.20,1562142060,0.71,71.07,0.81,rain,0.05,310.30,0.01,0.02,1562180640.00,0.58,rain,1012.30,Humid and mostly cloudy throughout the day.,1562149380,1562203140,86.97,1562183640,66.70,1562233500,86.97,1562183640,71.38,1562142180,1562126400,6,1562180040,9.98,224,13.84,1562129760,5.07,Marker1
3,3,91.30,1562279820,72.61,1562319840,91.30,1562279820,68.48,1562233560,0.21,69.25,0.77,rain,0.09,309.40,0.00,0.02,1562288580.00,0.43,rain,1014.60,Clear throughout the day.,1562235780,1562289540,88.05,1562270040,70.58,1562319900,88.05,1562270040,66.70,1562233500,1562212800,9,1562260680,10.00,216,13.00,1562212800,5.11,Marker1
4,4,94.56,1562352480,72.12,1562399700,94.56,1562352480,72.61,1562319840,0.29,72.78,0.83,rain,0.12,305.70,0.01,0.02,1562354160.00,0.51,rain,1016.00,Humid and partly cloudy throughout the day.,1562322240,1562375940,87.86,1562353620,69.98,1562399760,87.86,1562353620,70.58,1562319900,1562299200,9,1562348940,10.00,220,11.27,1562338980,5.34,Marker1


In [7]:
darksy_df['date_time'] = pd.to_datetime(darksy_df['time'],unit='s')
darksy_df['date_time']= darksy_df['date_time'].astype(str)
 
#shift data from gmt to est time
def date_convert(date):
    date = datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
    my_timestamp = date # some timestamp
    old_timezone = pytz.timezone("GMT")
    new_timezone = pytz.timezone("US/Eastern")

    # returns datetime in the new timezone
    my_timestamp_in_new_timezone = old_timezone.localize(my_timestamp).astimezone(new_timezone)
    return pd.to_datetime(my_timestamp_in_new_timezone)
 

darksy_df['date_time'] = darksy_df.apply(lambda x: date_convert(x['date_time']),axis=1)
darksy_df['Date'] = darksy_df['date_time'].dt.date
darksy_df['Hour'] = darksy_df['date_time'].dt.hour

In [8]:
darksy_df.head()

,Unnamed: 0,apparentTemperatureHigh,apparentTemperatureHighTime,apparentTemperatureLow,apparentTemperatureLowTime,apparentTemperatureMax,apparentTemperatureMaxTime,apparentTemperatureMin,apparentTemperatureMinTime,cloudCover,dewPoint,humidity,icon,moonPhase,ozone,precipIntensity,precipIntensityMax,precipIntensityMaxTime,precipProbability,precipType,pressure,summary,sunriseTime,sunsetTime,temperatureHigh,temperatureHighTime,temperatureLow,temperatureLowTime,temperatureMax,temperatureMaxTime,temperatureMin,temperatureMinTime,time,uvIndex,uvIndexTime,visibility,windBearing,windGust,windGustTime,windSpeed,Location,date_time,Date,Hour
0,0,94.20,1562018940,73.56,1562060280,94.20,1562018940,67.20,1561971900,0.22,69.16,0.78,rain,0.97,301.40,0.00,0.02,1562029440.00,0.33,rain,1015.40,Humid throughout the day.,1561976520,1562030400,87.97,1562018340,71.61,1562060040,87.97,1562018340,65.74,1561971840,1561953600,9,1562002920,10.00,287,17.53,1561993560,6.72,Marker1,2019-07-01 00:00:00-04:00,2019-07-01,0
1,1,95.30,1562091900,73.20,1562142060,95.30,1562091900,73.56,1562060280,0.19,71.76,0.76,rain,0.01,306.60,0.01,0.02,1562072760.00,0.50,rain,1013.00,Humid throughout the day.,1562062920,1562116740,90.19,1562094900,71.38,1562142180,90.19,1562094900,71.61,1562060040,1562040000,9,1562091000,10.00,243,16.72,1562107920,7.84,Marker1,2019-07-02 00:00:00-04:00,2019-07-02,0
2,2,91.00,1562183160,68.48,1562233560,91.00,1562183160,73.20,1562142060,0.71,71.07,0.81,rain,0.05,310.30,0.01,0.02,1562180640.00,0.58,rain,1012.30,Humid and mostly cloudy throughout the day.,1562149380,1562203140,86.97,1562183640,66.70,1562233500,86.97,1562183640,71.38,1562142180,1562126400,6,1562180040,9.98,224,13.84,1562129760,5.07,Marker1,2019-07-03 00:00:00-04:00,2019-07-03,0
3,3,91.30,1562279820,72.61,1562319840,91.30,1562279820,68.48,1562233560,0.21,69.25,0.77,rain,0.09,309.40,0.00,0.02,1562288580.00,0.43,rain,1014.60,Clear throughout the day.,1562235780,1562289540,88.05,1562270040,70.58,1562319900,88.05,1562270040,66.70,1562233500,1562212800,9,1562260680,10.00,216,13.00,1562212800,5.11,Marker1,2019-07-04 00:00:00-04:00,2019-07-04,0
4,4,94.56,1562352480,72.12,1562399700,94.56,1562352480,72.61,1562319840,0.29,72.78,0.83,rain,0.12,305.70,0.01,0.02,1562354160.00,0.51,rain,1016.00,Humid and partly cloudy throughout the day.,1562322240,1562375940,87.86,1562353620,69.98,1562399760,87.86,1562353620,70.58,1562319900,1562299200,9,1562348940,10.00,220,11.27,1562338980,5.34,Marker1,2019-07-05 00:00:00-04:00,2019-07-05,0


In [9]:
darksy_df.Location.unique()

array(['Marker1', 'Marker2', 'Marker3', 'Marker4', 'Marker5', 'Marker6',
       'Marker7', 'Marker8', 'Marker9', 'Marker10', 'Marker11',
       'Marker12', 'Marker13', 'Marker14', 'Marker15', 'Marker16',
       'Marker17', 'Marker18', 'Marker19', 'Marker20'], dtype=object)

In [10]:
darksy_df.to_csv('Darksky_All_Marker_July2019.csv')